In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

# PCA
from sklearn.decomposition import PCA


# Carrega a função SMOTE
import imblearn
from imblearn.over_sampling import SMOTE

# Ensemble Classifiers
from sklearn.ensemble import VotingClassifier, BaggingClassifier

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, RocCurveDisplay
from sklearn.metrics import accuracy_score


from sklearn.pipeline import Pipeline

%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Functions
def plot_sklearn_roc_curve(y_real, y_pred, auc, model_name):
    fpr, tpr, _ = roc_curve(y_real, y_pred)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax)
    plt.plot([0, 1], [0, 1], color = 'g',label="AUC="+str(round(auc,2)))
    plt.title('AUC: ' + model_name)
    plt.legend(loc=4, fontsize='xx-large')
    

    


## Caregando os dados e Primeiras impressões

In [ ]:
# Carrega os dados
dados = pd.read_csv('../input/indian-liver-patient-records/indian_liver_patient.csv')

In [ ]:
dados.head()

In [ ]:
dados.shape

In [ ]:
dados.columns

In [ ]:
dados.dtypes

In [ ]:
dados.isnull().sum()

In [ ]:
# Dados duplicados

dados_duplicados = dados[dados.duplicated(keep = False)]

dados_duplicados

In [ ]:
dados_duplicados.shape

#### Análise:

- Com exceção da variave 'Gender' todas as outras se apresetam na forma númerica
- Há poucos ocorrência de dados Vazios (Nas) no conjunto
- Existem duas variáveis com Nomes similares ('Total_Bilirubin' e 'Direct_Bilirubin'): verificar se são redundantes. 

## Análise Exporatória

### Variáveis Numéricas

In [ ]:
dados.describe()

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(15, 15), constrained_layout=True)

for ax, column in zip(axs.flat, dados.loc[:, dados.dtypes != 'object'].columns):
    ax.set_title(f'{column}', fontsize=14);
    sns.histplot(data=dados, x=column, kde=True, bins=10,color="darkblue", ax=ax);
    ax.set_xlabel(xlabel='');

 #### Análise
 
 - Diversas variáveis possuem inclinação da distribuição positiva, o que pode vir a ser um problema ao treinar o modelo de dados, bem como pode indicar a presença de outliers.
 - Podemos observar que a variável target está possivelmente desbalanceada o que pode vir a ser um problema.

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(15, 15), constrained_layout=True)

for ax, column in zip(axs.flat, dados.loc[:, dados.dtypes != 'object'].columns):
    ax.set_title(f'{column}', fontsize=14);
    sns.boxenplot(data=dados, x=column,color="skyblue", ax=ax);
    ax.set_xlabel(xlabel='');

#### Análise

- Com os Boxenplot podemos observar a presença de Outliers em diversas variáveis

In [ ]:
dados_corr = dados.corr()

# corr matrix
dados_corr

In [ ]:
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(dados_corr, dtype=bool))

# fig size
plt.figure(figsize=(15,15))

# ploting heeatmap
sns.heatmap(dados_corr, mask=mask,
            cmap='Blues', center=0,
            annot=True, square=True,  fmt='.2f',
            linewidths=.5, cbar_kws={"shrink": .5});

#### Análise

- Existem Três combinações de variáveis que possuem correlação maior que 70%, vae verificar se não estão reproduzindo a mesma informação.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

hues = ['Gender', 'Dataset']

for ax, column in zip(axs.flat, hues):
    sns.scatterplot(data=dados,
                    x='Total_Bilirubin',
                    y='Direct_Bilirubin',
                    hue=column,
                    ax=ax);




In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

for ax, column in zip(axs.flat, hues):
    sns.scatterplot(data=dados, x='Albumin', y='Total_Protiens', hue=column, ax=ax);

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

for ax, column in zip(axs.flat, hues):
    sns.scatterplot(data=dados,
                    x='Aspartate_Aminotransferase',
                    y='Alamine_Aminotransferase',
                    hue=column,
                    ax=ax);


#### Análise

- Nas três variáveis com alta correlação podemos observar que o Gênero não tem muita influência, bem como duas dessas variáveis podem representar informação dupicada;
- Vale Realizar a exclusão das mesma no treinamento e verificar como isso afeta o resultado. 

### Variáveis Categóricas

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

for ax, column in zip(axs.flat, hues):
    ax = sns.countplot(data=dados, x=column, ax=ax)
    
    # Display bar count
    ax.bar_label(ax.containers[0]);
    # Setting tittle
    ax.set_title(f'Count plot \n Variable: {column}')

#### Análise

- Podemos ver que a variável target está reamennte desbalanceada, o que pode ocasionar problemas no treinamento dos dados

## Limpeza

### Tratamento de dados ausentes e dupicados

    Como os dados há apenas 4 valores ausentes é mais vantajoso remorelos dos conjunto, o mesmo vale para os dados duplicados como existem somente 11 registros o melhor é removê-los.

In [ ]:
# Removendo Dados ausentes
dados = dados.dropna(how = 'any')

In [ ]:
# Removendo dados duplicados
dados = dados.drop_duplicates()

In [ ]:
dados.shape

In [ ]:
dados.sample(10)

## Pré-processamento

   - Nesta fase iremos tratar as variáveis 'Gender' e 'Dataset' visto que não estão em conformidade para o treinamento dos dados;
   - Divisão de treino e teste;
   - Realizaremos o balanceamento da variável alvo;
   - Trataremos dos outliers
   - Padronização dos dados
   

### Variáveis Gender e Dataset

- Em relação a variável 'Gender' iremos mudar de string para uma variáve numérica a fim de possibilitar o treinamento;
- Em relação a variável 'Dataset' iremos alterar o nome para 'target' e o range de 1-2 para 0-1 a fim de padronizar a variáve alvo.

In [ ]:
# Variável Gender
dados['Gender'] = np.where(dados['Gender'] == 'Female', 0, 1)

# Variável Dataset
dados['Target'] = np.where(dados['Dataset'] == 2, 0, 1)

dados = dados.drop(labels=['Dataset'], axis=1)

In [ ]:
dados.sample(5)

### Remoção automática de outiers

In [ ]:
# Cria um objeto separado para a variável target
y = dados.Target.values

# Cria um objeto separadado para as variáveis de entrada
X = dados.drop('Target', axis = 1).values

# Grava nome das colunas dos dados
X_columns = dados.drop('Target', axis = 1).columns


# Random Seed
seed=42

iso = IsolationForest(random_state=seed)

yhat = iso.fit_predict(X)
# select all rows that are not outliers
mask = yhat != -1
X, y = X[mask, :], y[mask]

In [ ]:
#print(X.shape, y.shape)

In [ ]:
pd.DataFrame(X, columns=X_columns).sample(10)

### Divisão de treino e teste

In [ ]:
# Split em dados de treino e teste com amostragem estratificada
X_treino, X_teste, y_treino, y_teste = train_test_split(X, 
                                                        y, 
                                                        test_size = 0.25, 
                                                        random_state = seed,
                                                        stratify = y)

# Print do shape
print(X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape)

### Balanceamento de Classe

In [ ]:
# Vamos aplicar a técnica de oversampling e aumentar o número de exemplos da classe minoritária
over_sampler = SMOTE(k_neighbors = 2, random_state=seed)

# Aplica o oversampling (deve ser feito somente com dados de treino)
X_res, y_res = over_sampler.fit_resample(X_treino, y_treino)

# Ajusta o nome do dataset de treino para X
X_treino = X_res

# Ajusta o nome do dataset de treino para y
y_treino = y_res


# Print do shape
print(X_treino.shape, y_treino.shape)

## **Machine Learn**

###  Construção, Treinamento e Avaliação do Modelo 1 com Regressão Logística (Benchmark)

In [ ]:
# Define lista de hiperparâmetros
tuned_params_v1 = {'logistic__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 
                   'logistic__penalty': [None, 'elasticnet', 'l1', 'l2']}

In [ ]:

# Padronização dos dados
scaler = StandardScaler()

# PCA para redução de dimensionalidade
pca = PCA(n_components=3, svd_solver='randomized')



# Criando o Pipeine
pipe_logistic = Pipeline([('scaler', scaler), ('pca', pca),('logistic', LogisticRegression(random_state=seed))])

# Score used in GridSearchCV 'roc_auc'  'recall'   'f1'
score = 'roc_auc'

# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
modelo_v1 = GridSearchCV(pipe_logistic, 
                         tuned_params_v1,
                         scoring = score,
                         cv = 10,
                         n_jobs = -1)




In [ ]:
# Treinamento do modelo
modelo_v1.fit(X_treino, y_treino)

In [ ]:
# Selecionamos o melhor modelo
modelo_v1.best_estimator_


In [ ]:
# Previsões com dados de teste
y_pred_v1 = modelo_v1.predict(X_teste)

In [ ]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v1 = modelo_v1.predict_proba(X_teste)

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v1)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});


In [ ]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v1).ravel()

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v1 = roc_auc_score(y_teste, y_pred_v1)
print(roc_auc_v1)

In [ ]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v1, tpr_v1, thresholds = roc_curve(y_teste, y_pred_proba_v1[:,1])

In [ ]:
# AUC em teste
auc_v1 = auc(fpr_v1, tpr_v1)
print(auc_v1)

In [ ]:
# Acurácia em teste
acuracia_v1 = accuracy_score(y_teste, y_pred_v1)
print(acuracia_v1)

In [ ]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v1 = {'Nome': 'modelo_v1', 
                  'Algoritmo': 'Regressão Logística', 
                  'ROC_AUC Score': roc_auc_v1,
                  'AUC Score': auc_v1,
                  'Acurácia': acuracia_v1}

In [ ]:
df_modelos = pd.DataFrame()

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v1, ignore_index = True)

In [ ]:
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v1[:, 1], auc_v1, dict_modelo_v1['Algoritmo']);


In [ ]:
display(df_modelos)

### RandomForest (Model V2)

In [ ]:
# Define lista de hiperparâmetros
tuned_params_v2 = {'rf__n_estimators': [100, 200, 300, 400, 500],
                   'rf__min_samples_split': [2, 5, 10],
                   'rf__min_samples_leaf': [1, 2, 4],
                   'rf__criterion': ["gini", "entropy", "log_loss"]}

In [ ]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
pipe_rf = Pipeline([('scaler', scaler), ('pca', pca),('rf', RandomForestClassifier(random_state=seed))])


modelo_v2 = GridSearchCV(pipe_rf, 
                         tuned_params_v2,  
                         scoring = score,
                         cv = 10,
                         n_jobs = -1)




In [ ]:
# Treinamento do modelo
modelo_v2.fit(X_treino, y_treino)

In [ ]:
# Selecionamos o melhor modelo
modelo_v2.best_estimator_


In [ ]:
# Previsões com dados de teste
y_pred_v2 = modelo_v2.predict(X_teste)

In [ ]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v2 = modelo_v2.predict_proba(X_teste)

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v2)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});


In [ ]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v2).ravel()

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v2 = roc_auc_score(y_teste, y_pred_v2)
print(roc_auc_v2)

In [ ]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v2, tpr_v2, thresholds = roc_curve(y_teste, y_pred_proba_v2[:,1])

In [ ]:
# AUC em teste
auc_v2 = auc(fpr_v2, tpr_v2)
print(auc_v2)

In [ ]:
# Acurácia em teste
acuracia_v2 = accuracy_score(y_teste, y_pred_v2)
print(acuracia_v2)

In [ ]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v2 = {'Nome': 'modelo_v2', 
                  'Algoritmo': 'RandomForest Classifier', 
                  'ROC_AUC Score': roc_auc_v2,
                  'AUC Score': auc_v2,
                  'Acurácia': acuracia_v2}

In [ ]:

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v2, ignore_index = True)

In [ ]:
#create ROC curve
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v2[:, 1], auc_v2, dict_modelo_v2['Algoritmo'])


In [ ]:
display(df_modelos)

### KNN (Model V3) 

In [ ]:
# Define lista de hiperparâmetros
tuned_params_v3 = {'knn__n_neighbors': np.arange(1, 30), 
                   'knn__weights': ["uniform", "distance"]}

In [ ]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
pipe_knn = Pipeline([('scaler', scaler), ('pca', pca),('knn', KNeighborsClassifier())])


modelo_v3 = GridSearchCV(pipe_knn, 
                         tuned_params_v3, 
                         scoring = score,
                         cv = 10,
                         n_jobs = -1)




In [ ]:
# Treinamento do modelo
modelo_v3.fit(X_treino, y_treino)

In [ ]:
# Selecionamos o melhor modelo
modelo_v3.best_estimator_


In [ ]:
# Previsões com dados de teste
y_pred_v3 = modelo_v3.predict(X_teste)

In [ ]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v3 = modelo_v3.predict_proba(X_teste)

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v3)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});


In [ ]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v3).ravel()

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v3 = roc_auc_score(y_teste, y_pred_v3)
print(roc_auc_v3)

In [ ]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v3, tpr_v3, thresholds = roc_curve(y_teste, y_pred_proba_v3[:,1])

In [ ]:
# AUC em teste
auc_v3 = auc(fpr_v3, tpr_v3)
print(auc_v3)

In [ ]:
# Acurácia em teste
acuracia_v3 = accuracy_score(y_teste, y_pred_v3)
print(acuracia_v3)

In [ ]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v3 = {'Nome': 'modelo_v3', 
                  'Algoritmo': 'KNeighbors Classifier', 
                  'ROC_AUC Score': roc_auc_v3,
                  'AUC Score': auc_v3,
                  'Acurácia': acuracia_v3}

In [ ]:

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v3, ignore_index = True)

In [ ]:
#create ROC curve
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v3[:, 1], auc_v3, dict_modelo_v3['Algoritmo'])


In [ ]:
display(df_modelos)

### SVM (Model V4)

In [ ]:
# Define lista de hiperparâmetros
tuned_params_v4 = {'svm__C': [0.001, 0.01, 0.1, 1, 10], 
                   'svm__gamma': [0.001, 0.01, 0.1, 1]}

In [ ]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
pipe_svm = Pipeline([('scaler', scaler), ('pca', pca),('svm', SVC(random_state=seed, probability=True))])


modelo_v4 = GridSearchCV(pipe_svm, 
                         tuned_params_v4,  
                         scoring = score,
                         cv = 10,
                         n_jobs = -1)




In [ ]:
# Treinamento do modelo
modelo_v4.fit(X_treino, y_treino)

In [ ]:
# Selecionamos o melhor modelo
modelo_v4.best_estimator_


In [ ]:
# Previsões com dados de teste
y_pred_v4 = modelo_v4.predict(X_teste)

In [ ]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v4 = modelo_v4.predict_proba(X_teste)

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v4)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});

In [ ]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v4).ravel()

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v4 = roc_auc_score(y_teste, y_pred_v4)
print(roc_auc_v4)

In [ ]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v4, tpr_v4, thresholds = roc_curve(y_teste, y_pred_proba_v4[:,1])

In [ ]:
# AUC em teste
auc_v4 = auc(fpr_v4, tpr_v4)
print(auc_v4)

In [ ]:
# Acurácia em teste
acuracia_v4 = accuracy_score(y_teste, y_pred_v4)
print(acuracia_v4)

In [ ]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v4 = {'Nome': 'modelo_v4', 
                  'Algoritmo': 'SVM Classifier', 
                  'ROC_AUC Score': roc_auc_v4,
                  'AUC Score': auc_v4,
                  'Acurácia': acuracia_v4}

In [ ]:

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v4, ignore_index = True)

In [ ]:
#create ROC curve
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v4[:, 1], auc_v4, dict_modelo_v4['Algoritmo'])


In [ ]:
display(df_modelos)

### Decision Tree (Model V5)

In [ ]:
# Define lista de hiperparâmetros
tuned_params_v5 = {'dt__min_samples_split': [2, 3, 4, 5, 7], 
                   'dt__min_samples_leaf': [1, 2, 3, 4, 6], 
                   'dt__max_depth': [2, 3, 4, 5, 6, 7]}

In [ ]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
pipe_knn = Pipeline([('scaler', scaler), ('pca', pca),('dt', DecisionTreeClassifier(random_state=seed))])


modelo_v5 = GridSearchCV(pipe_knn, 
                         tuned_params_v5, 
                         scoring = score,
                         cv = 10,
                         n_jobs = -1)




In [ ]:
# Treinamento do modelo
modelo_v5.fit(X_treino, y_treino)

In [ ]:
# Selecionamos o melhor modelo
modelo_v5.best_estimator_


In [ ]:
# Previsões com dados de teste
y_pred_v5 = modelo_v5.predict(X_teste)

In [ ]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v5 = modelo_v5.predict_proba(X_teste)

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v5)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});

In [ ]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v5).ravel()

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v5 = roc_auc_score(y_teste, y_pred_v5)
print(roc_auc_v5)

In [ ]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v5, tpr_v5, thresholds = roc_curve(y_teste, y_pred_proba_v5[:,1])

In [ ]:
# AUC em teste
auc_v5 = auc(fpr_v5, tpr_v5)
print(auc_v5)

In [ ]:
# Acurácia em teste
acuracia_v5 = accuracy_score(y_teste, y_pred_v5)
print(acuracia_v5)

In [ ]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v5 = {'Nome': 'modelo_v5', 
                  'Algoritmo': 'DecisionTree Classifier', 
                  'ROC_AUC Score': roc_auc_v5,
                  'AUC Score': auc_v5,
                  'Acurácia': acuracia_v5}

In [ ]:

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v5, ignore_index = True)

In [ ]:
#create ROC curve
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v5[:, 1], auc_v5, dict_modelo_v5['Algoritmo'])


In [ ]:
display(df_modelos)

### Bagging Classifier (Model V6)

In [ ]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
knn_params= modelo_v3.get_params()

knn = KNeighborsClassifier(n_neighbors=knn_params['estimator__knn__n_neighbors'],
                           weights=knn_params['estimator__knn__weights'])


bagging = BaggingClassifier(knn,
                            n_estimators=20,
                            random_state=seed)


modelo_v6 = Pipeline([('scaler', scaler), ('pca', pca), ('bagging', bagging)])

In [ ]:
# Treinamento do modelo
modelo_v6.fit(X_treino, y_treino)

In [ ]:
# Previsões com dados de teste
y_pred_v6 = modelo_v6.predict(X_teste)

In [ ]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v6 = modelo_v6.predict_proba(X_teste)

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v6)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});

In [ ]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v6).ravel()

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v6 = roc_auc_score(y_teste, y_pred_v6)
print(roc_auc_v6)

In [ ]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v6, tpr_v6, thresholds = roc_curve(y_teste, y_pred_proba_v6[:,1])

In [ ]:
# AUC em teste
auc_v6 = auc(fpr_v6, tpr_v6)
print(auc_v6)

In [ ]:
# Acurácia em teste
acuracia_v6 = accuracy_score(y_teste, y_pred_v6)
print(acuracia_v6)

In [ ]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v6 = {'Nome': 'modelo_v6', 
                  'Algoritmo': 'Bagging Classifier', 
                  'ROC_AUC Score': roc_auc_v6,
                  'AUC Score': auc_v6,
                  'Acurácia': acuracia_v6}

In [ ]:

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v6, ignore_index = True)

In [ ]:
#create ROC curve
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v6[:, 1], auc_v6, dict_modelo_v6['Algoritmo'])


In [ ]:
display(df_modelos)

### Voting Classifier (Model V7)

In [ ]:
# Define lista de hiperparâmetros
lr_params = modelo_v1.get_params()
rf_params = modelo_v2.get_params()

In [ ]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros

lr = LogisticRegression(random_state=seed,
                        C=lr_params['estimator__logistic__C'],
                        penalty=lr_params['estimator__logistic__penalty'])

rf = RandomForestClassifier(random_state=seed,
                            n_estimators=rf_params['estimator__rf__n_estimators'],
                            min_samples_split=rf_params['estimator__rf__min_samples_split'],
                            min_samples_leaf=rf_params['estimator__rf__min_samples_leaf'],
                            criterion=rf_params['estimator__rf__criterion'])





modelo_v7 = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('clf', VotingClassifier([
        ('logistic', lr),
        ('rf', rf),
        ('knn', knn)],
        voting='soft')),
])




In [ ]:
# Treinamento do modelo
modelo_v7.fit(X_treino, y_treino)

In [ ]:
# Previsões com dados de teste
y_pred_v7 = modelo_v7.predict(X_teste)

In [ ]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v7 = modelo_v7.predict_proba(X_teste)

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v7)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});

In [ ]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v7).ravel()

In [ ]:
print(tn, fp, fn, tp)

In [ ]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v7 = roc_auc_score(y_teste, y_pred_v7)
print(roc_auc_v7)

In [ ]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v7, tpr_v7, thresholds = roc_curve(y_teste, y_pred_proba_v7[:,1])

In [ ]:
# AUC em teste
auc_v7 = auc(fpr_v7, tpr_v7)
print(auc_v7)

In [ ]:
# Acurácia em teste
acuracia_v7 = accuracy_score(y_teste, y_pred_v7)
print(acuracia_v7)

In [ ]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v7 = {'Nome': 'modelo_v7', 
                  'Algoritmo': 'Voting Classifier', 
                  'ROC_AUC Score': roc_auc_v7,
                  'AUC Score': auc_v7,
                  'Acurácia': acuracia_v7}

In [ ]:

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v7, ignore_index = True)

In [ ]:
#create ROC curve
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v7[:, 1], auc_v7, dict_modelo_v7['Algoritmo'])


In [ ]:
display(df_modelos)

### Escolha do Modelo

Neste projeto o Modelo fina foi escolhido com base no AUC de cada versão. O Modelo com melhor desenpenho foi o Votig Classifier (composto por KNN, Logistic Regresson e Random Florest).

In [ ]:
df_modelos[df_modelos['AUC Score'] == df_modelos['AUC Score'].max()]

In [ ]:
# Matriz de confusão
cf_matrix = confusion_matrix(y_teste, y_pred_v7)

# fig size
plt.figure(figsize=(6,6))

# Plot
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={'fontsize':'xx-large'});

In [ ]:
#create ROC curve
plot_sklearn_roc_curve(y_teste, y_pred_proba_v7[:, 1], auc_v7, dict_modelo_v7['Algoritmo'])
